### Author: Chunru Zheng (cz8yb@virginia.edu) 
### Class: DS 5001 Spring 2023

In [2]:
import pandas as pd 
import numpy as np
import copy 
import random
from matplotlib import pyplot as plt 
from glob import glob
import re
import nltk
from numpy.linalg import norm
from scipy.spatial.distance import pdist
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stop = stopwords.words('english')
from nltk import pos_tag
import seaborn as sns; sns.set()

In [3]:
rawdata = "/Users/Christine/Library/Mobile Documents/com~apple~CloudDocs/Patent/rawdata"
workdata = "/Users/Christine/Library/Mobile Documents/com~apple~CloudDocs/Patent/workdata"

In [19]:
#pooling_patent_data = pd.read_csv(f'{workdata}/pooling_patent_data.csv')
#PAIRS = pd.read_csv(f'{workdata}/PAIRS.csv')

In [4]:
##############################################
# try on a small sample, don't change any name: 
# pooling_patent_data = pooling_patent_data.sample(200)
# PAIRS = PAIRS.sample(200)
##############################################
# Add some summary statistics. 

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Apply stemming and remove punctuation, digits, and non-English words
stemmer = PorterStemmer()

def clean_and_stem_nouns_verbs(sentence):
    # Remove punctuation and digits
    clean_sentence = re.sub(r'[^\w\s]', ' ', sentence)
    clean_sentence = re.sub(r'\d+', '', clean_sentence)
    
    # Tokenize the sentence
    tokens = word_tokenize(clean_sentence)
    
    # POS tagging and filtering nouns (NN) and verbs (VB)
    pos_tags = pos_tag(tokens)
    filtered_tokens = [token for token, pos in pos_tags if pos.startswith('NN') or pos.startswith('VB')]
    
    # Stem and filter non-English words
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens if token.isalpha() and token.isascii()]
    
    return ' '.join(stemmed_tokens)

pooling_patent_data['cleaned_and_stemmed_nouns_verbs_abstract'] = pooling_patent_data['Patent_abstract'].astype(str).apply(clean_and_stem_nouns_verbs)
pooling_patent_data.to_csv(f'{workdata}/pooling_patent_data_filter.csv', index= False)

In [4]:
pooling_patent_data = pd.read_csv(f'{workdata}/pooling_patent_data_filter.csv')
pooling_patent_data = pooling_patent_data.dropna(subset=['cleaned_and_stemmed_nouns_verbs_abstract'])

In [5]:
pooling_patent_data

,Patent_ID,Patent_abstract,cleaned_and_stemmed_nouns_verbs_abstract
0,CN98118142.2,A Chinese patent medicine in the form of capsu...,patent medicin form capsul is prepar chiretta ...
1,CN200710126165.3,The invention discloses a method and apparatus...,invent disclos method apparatu hide data packa...
2,CN200610060717.0,The method comprises: receiving an audio signa...,method compris receiv signal signal sourc get ...
3,CN00103340.9,A device for eliminating the mini ice crystal ...,devic elimin mini ice crystal supercool water ...
4,CN200710075612.7,The invention provides a method of producing s...,invent provid method produc effect calcul part...
...,...,...,...
263667,JPH0217849,PURPOSE: To facilitate stopping rotation of a ...,purpos facilit stop rotat rotor open hole posi...
263668,WO2005017670A,"An embodiment of the invention, for controllin...",embodi invent control context menu use web pag...
263669,DE202005008280U,A radial fan (20) has an impeller (22) enclose...,fan ha impel enclos case air inlet air inlet o...
263670,KR100211453B,AbstractTranslated from Korean The remote cont...,abstracttransl korean control lock unlock devi...


## TFIDF_table

In [6]:
# Create a TfidfVectorizer instance with the specified parameters
##tfidf_table:  max_df=0.95, min_df=2: 38.44G with 50,000 cols
##tfidf_table_filter:  max_df=0.95, min_df = 0.001: 2G with 2605 cols(this one)
##tfidf_table_filter:  max_df=0.95, min_df = 0.005: 7M with 100 cols

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=0.001)

# Fit and transform the 'cleaned_and_stemmed_nouns_verbs_abstract' column into a TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(pooling_patent_data['cleaned_and_stemmed_nouns_verbs_abstract'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_table = pd.DataFrame(tfidf_matrix.toarray(), index=pooling_patent_data['Patent_ID'], columns=vectorizer.get_feature_names_out())
tfidf_table.to_csv(f'{workdata}/tfidf_table_filter.csv', index= True)
tfidf_table

,aaa,ab,abil,abnorm,abov,abras,absenc,absorb,absorpt,abstract,...,yield,yoke,zero,zinc,zirconium,zn,zno,zone,zoom,zr
Patent_ID,,,,,,,,,,,,,,,,,,,,,
CN98118142.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN200710126165.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN200610060717.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN00103340.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN200710075612.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JPH0217849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WO2005017670A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DE202005008280U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074061,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#tfidf_table = pd.read_csv(f'{workdata}/tfidf_table_filter.csv').set_index('Patent_ID')

## 1) PCA 

In [7]:
from sklearn.decomposition import PCA

# Number of principal components to keep
n_components = 30

# Fit PCA to the tfidf_table
pca = PCA(n_components=n_components)
pca_data = pca.fit_transform(tfidf_table)

# Convert the PCA data to a DataFrame
pca_columns = [f'PCA{i}' for i in range(1, n_components + 1)]
pca_table = pd.DataFrame(pca_data, index=tfidf_table.index, columns=pca_columns)
pca_table.to_csv(f'{workdata}/pca_table.csv', index= True)
pca_table

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA21,PCA22,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30
Patent_ID,,,,,,,,,,,,,,,,,,,,,
CN98118142.2,-0.108765,-0.115050,-0.139575,-0.136057,-0.035418,-0.032474,-0.000578,-0.046746,-0.037284,-0.029182,...,-0.010649,0.024430,-0.008294,-0.028358,0.020210,0.010533,0.037416,-0.179239,0.032331,-0.001662
CN200710126165.3,0.090579,0.013088,-0.084545,-0.003759,0.243054,-0.005952,0.126249,-0.127853,0.044727,-0.001630,...,-0.002702,-0.014570,-0.070497,-0.034042,0.012911,0.105378,-0.023400,0.055432,-0.090239,0.008461
CN200610060717.0,0.045659,0.092544,-0.099450,-0.046133,0.010064,0.039879,-0.060000,0.011216,-0.004933,0.071896,...,0.028719,0.014377,-0.019554,-0.013400,0.006667,0.036516,-0.027494,0.013227,0.019030,0.059932
CN00103340.9,-0.114750,0.004335,0.086867,-0.087187,-0.006442,-0.007243,0.090192,0.047516,0.015591,-0.003035,...,0.015878,-0.025255,0.035511,0.060307,-0.009201,0.024324,0.106407,-0.019073,-0.053359,-0.014539
CN200710075612.7,0.036084,0.141411,-0.065228,-0.033701,-0.024967,0.091680,-0.066200,-0.009231,-0.054860,0.135335,...,0.044258,0.004947,-0.042890,-0.000931,-0.010058,-0.076068,0.028275,-0.042254,0.067653,0.002871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JPH0217849,-0.133285,0.041886,0.218574,0.014590,0.068096,0.046837,-0.050481,-0.113412,0.016304,-0.010842,...,-0.016790,0.006702,-0.036098,0.015423,0.031460,0.015166,0.059103,-0.033766,0.005196,-0.029307
WO2005017670A,0.023023,-0.029968,-0.009605,-0.011357,0.004827,-0.019152,-0.061985,0.021958,0.013398,-0.035157,...,0.007173,0.024039,0.007920,-0.080108,-0.043535,-0.018012,-0.020851,-0.009422,-0.038695,-0.004104
DE202005008280U,-0.121668,0.130797,0.125934,-0.058424,-0.056817,0.037031,0.147934,0.123884,0.058791,-0.001525,...,0.085808,-0.033054,0.124244,-0.123089,0.005782,0.170870,-0.065131,-0.021199,0.018580,0.022430


In [ ]:
#pd.read_csv(f'{workdata}/pca_table.csv').set_index('Patent_ID')

## 2) LDA: Topic Model

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

# Fit LDA to the tfidf_table
lda = LatentDirichletAllocation(n_components=30)
lda.fit(tfidf_table)

# Transform the tfidf_table to topic distribution
topic_distribution = lda.transform(tfidf_table)

# Create a DataFrame with Patent_ID and topic distributions
topic_distribution_df = pd.DataFrame(topic_distribution, columns=[f'topic_{i}' for i in range(topic_distribution.shape[1])])

topic_distribution_df['Patent_ID'] = pooling_patent_data.reset_index()['Patent_ID']
topic_distribution_df = topic_distribution_df.reset_index(drop=True).set_index('Patent_ID')
topic_distribution_df.to_csv(f'{workdata}/lda_table.csv', index= True)
topic_distribution_df

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
Patent_ID,,,,,,,,,,,,,,,,,,,,,
CN98118142.2,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,...,0.830204,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855,0.005855
CN200710126165.3,0.006101,0.006101,0.006101,0.006101,0.006101,0.006101,0.006101,0.006101,0.006101,0.006101,...,0.006101,0.006101,0.006101,0.006101,0.823084,0.006101,0.006101,0.006101,0.006101,0.006101
CN200610060717.0,0.007346,0.007346,0.007346,0.007346,0.007346,0.007346,0.007346,0.007346,0.007346,0.007346,...,0.007346,0.007346,0.007346,0.007346,0.786970,0.007346,0.007346,0.007346,0.007346,0.007346
CN00103340.9,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,...,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189,0.006189
CN200710075612.7,0.005773,0.201858,0.005773,0.005773,0.005773,0.005773,0.005773,0.005773,0.005773,0.005773,...,0.005773,0.005773,0.005773,0.005773,0.636496,0.005773,0.005773,0.005773,0.005773,0.005773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JPH0217849,0.005522,0.005522,0.005522,0.005522,0.005522,0.005522,0.005522,0.005522,0.005522,0.005522,...,0.005522,0.005522,0.005522,0.005522,0.005522,0.005522,0.420542,0.005522,0.005522,0.005522
WO2005017670A,0.008320,0.758724,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320,...,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320,0.008320
DE202005008280U,0.006775,0.006775,0.006775,0.006775,0.031201,0.006775,0.006775,0.006775,0.006775,0.006775,...,0.006775,0.006775,0.006775,0.006775,0.006775,0.006775,0.006775,0.006775,0.006775,0.006775


In [10]:
# pd.read_csv(f'{workdata}/lda_table.csv').set_index('Patent_ID')

In [11]:
# Display the top words for each topic
n_top_words = 10

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(" ".join([tfidf_table.columns[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print("\n")

Topic #1:
compound formula group acid composit alkyl prepar salt contain atom


Topic #2:
display program devic user inform comput imag control screen input


Topic #3:
mold roll steel materi surfac layer plate weld form die


Topic #4:
user authent inform network server key access servic content devic


Topic #5:
board connector card circuit cabl connect contact wire termin conductor


Topic #6:
fiber yarn fabric protein paper invent method dye gene spin


Topic #7:
layer substrat electrod film form semiconductor surfac region metal materi


Topic #8:
signal antenna frequenc band line transmiss wavelength receiv unit fiber


Topic #9:
station network commun base wireless radio inform termin cell locat


Topic #10:
water fluid wash chamber ga liquid tank flow pump filter


Topic #11:
product acid water prepar process reaction solut materi invent method


Topic #12:
light imag emit surfac beam display sourc reflect color plate


Topic #13:
manag data file server network inform client so

from nltk.corpus import stopwords
from collections import Counter
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
## 3) Word Embedding: vector space 

In [12]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def tokenize_and_filter(sentence, min_df=0.005, max_df=0.9):
    clean_sentence = re.sub(r'[^\w\s]', ' ', sentence)
    clean_sentence = re.sub(r'\d+', '', clean_sentence)
    tokens = word_tokenize(clean_sentence)
    lower_tokens = [token.lower() for token in tokens if token.isalpha() and token.isascii() and token.lower() not in stop_words]

    # Filter out words that are too frequent or too rare
    token_counts = Counter(lower_tokens)
    filtered_tokens = [token for token, count in token_counts.items() if count >= min_df and count / len(lower_tokens) <= max_df]

    return filtered_tokens

pooling_patent_data['filtered_tokenized_abstract'] = pooling_patent_data['cleaned_and_stemmed_nouns_verbs_abstract'].astype(str).apply(tokenize_and_filter)

def average_word_embedding(tokens, model, vector_size):
    embeddings = [model.wv[token] for token in tokens if token in model.wv.key_to_index]
    if len(embeddings) == 0:
        return np.zeros(vector_size)
    return np.mean(embeddings, axis=0)

# Train a Word2Vec model on the filtered tokenized abstracts
word2vec_model = Word2Vec(sentences=pooling_patent_data['filtered_tokenized_abstract'], vector_size=100, window=5, min_count=1, workers=4)

# Save the trained Word2Vec model
word2vec_model.save(f"{workdata}/word2vec_model_filtered.model")

# Compute the average word embeddings for each filtered abstract
pooling_patent_data['avg_word_embedding_filtered'] = pooling_patent_data['filtered_tokenized_abstract'].apply(lambda x: average_word_embedding(x, word2vec_model, 100))
avg_word_embedding_filtered = pooling_patent_data[['Patent_ID','avg_word_embedding_filtered']]

avg_word_embedding_filtered.to_csv(f'{workdata}/wbed_table.csv', index= False)

# Create a new DataFrame from the lists
expanded_df = pd.DataFrame(avg_word_embedding_filtered['avg_word_embedding_filtered'].tolist())

# Set the index of the expanded_df to be the same as the index of avg_word_embedding_filtered
expanded_df.index = avg_word_embedding_filtered.index

# Rename the columns of the new DataFrame as 'Vec1', 'Vec2', etc.
expanded_df.columns = [f'Vec{i+1}' for i in range(expanded_df.shape[1])]

# Drop the 'avg_word_embedding_filtered' column from the original DataFrame
avg_word_embedding_filtered = avg_word_embedding_filtered.drop('avg_word_embedding_filtered', axis=1)

# Concatenate the original DataFrame with the new DataFrame along the columns axis
wbed_table = pd.concat([avg_word_embedding_filtered, expanded_df], axis=1)
wbed_table.index.name = 'Patent_ID'
wbed_table.to_csv(f'{workdata}/wbed_table.csv', index= True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Christine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Patent_ID,avg_word_embedding_filtered
0,CN98118142.2,"[0.0052745203, 1.0487688, 0.6639454, 0.3858861..."
1,CN200710126165.3,"[-0.29572046, 0.3397918, 0.07396194, 0.0764177..."
2,CN200610060717.0,"[-0.24969853, 0.43460408, -0.40595564, -0.2730..."
3,CN00103340.9,"[-0.19181477, -0.24820161, -0.26869848, -0.055..."
4,CN200710075612.7,"[0.07892086, 0.38816518, -0.014554849, -0.3827..."
...,...,...
263667,JPH0217849,"[-0.12599207, 0.7761822, -0.40270397, 0.006816..."
263668,WO2005017670A,"[-0.44802254, 0.056004073, -0.22580059, -0.362..."
263669,DE202005008280U,"[0.11922814, -0.15174168, -0.461393, -0.018574..."
263670,KR100211453B,"[-0.38021043, 0.4701905, -0.36553308, -0.48908..."


## Calculate Distance

In [70]:
def get_distance(pair, table, distance_type='euclidean'):
    patent1_id = pair['APPLN_ID_SIPO']
    patent2_id = pair['PubNum_google']
    

    if patent1_id not in table.index or patent2_id not in table.index:
        return float('inf')  # Return a large distance value if either key is missing
    
    patent1_features = table.loc[patent1_id].values
    patent2_features = table.loc[patent2_id].values


    if distance_type == 'euclidean':
        return euclidean(patent1_features, patent2_features)
    elif distance_type == 'cosine':
        return cosine(patent1_features, patent2_features)
    elif distance_type == 'jensenshannon':
        return jensenshannon(patent1_features, patent2_features)
    else:
        raise ValueError(f"Unsupported distance type: {distance_type}")

In [65]:
pca_table = pd.read_csv(f'{workdata}/pca_table.csv').set_index('Patent_ID')
wbed_table = pd.read_csv(f'{workdata}/wbed_table.csv').set_index('Patent_ID')
lda_table = pd.read_csv(f'{workdata}/lda_table.csv').set_index('Patent_ID')
PAIRS = pd.read_csv(f'{workdata}/PAIRS.csv')

In [71]:
from scipy.spatial.distance import euclidean, cosine, jensenshannon

distance_df = pd.DataFrame(columns=['APPLN_ID_SIPO', 'PubNum_google', 
                                    'distance_lda_euclidean', 'distance_lda_cosine', 'distance_lda_jensenshannon',
                                    'distance_pca_euclidean', 'distance_pca_cosine', 'distance_pca_jensenshannon',
                                    'distance_wbed_euclidean', 'distance_wbed_cosine', 'distance_wbed_jensenshannon'])

for _, pair in PAIRS.iterrows():
    tables = [lda_table, pca_table, wbed_table]
    distances = ['euclidean', 'cosine', 'jensenshannon']
    results = {}

    for table_name, table in zip(['lda', 'pca', 'wbed'], tables):
        for distance_type in distances:
            distance = get_distance(pair, table, distance_type=distance_type)
            results[f'distance_{table_name}_{distance_type}'] = distance

    result_row = pd.DataFrame({
        'APPLN_ID_SIPO': [pair['APPLN_ID_SIPO']],
        'PubNum_google': [pair['PubNum_google']],
        **results
    })

    distance_df = pd.concat([distance_df, result_row], ignore_index=True)
    
distance_df.to_csv(f'{workdata}/distance_df.csv', index=False)
distance_df

/Users/Christine/opt/anaconda3/lib/python3.9/site-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/Users/Christine/opt/anaconda3/lib/python3.9/site-packages/scipy/spatial/distance.py:1291: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)


,APPLN_ID_SIPO,PubNum_google,distance_lda_euclidean,distance_lda_cosine,distance_lda_jensenshannon,distance_pca_euclidean,distance_pca_cosine,distance_pca_jensenshannon,distance_wbed_euclidean,distance_wbed_cosine,distance_wbed_jensenshannon
0,CN98118142.2,JP2897659B,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,CN98118142.2,US5644677,1.158632,0.983931,0.735256,0.49571,0.987218,inf,8.046884,0.894533,inf
2,CN98118142.2,KR100188089B,1.160848,0.984116,0.736094,0.516308,0.99796,inf,7.24412,1.0195,inf
3,CN98118142.2,US5572591,1.133489,0.981703,0.726175,0.847552,1.028149,inf,7.978792,0.882985,inf
4,CN98118142.2,JPH09322078,1.164768,0.98444,0.73759,0.672208,1.088952,inf,9.08189,1.106969,inf
...,...,...,...,...,...,...,...,...,...,...,...
303540,CN200710023249.4,CN2676102Y,0.384796,0.104343,0.31478,0.391238,0.411181,inf,2.306281,0.074339,inf
303541,CN200710023249.4,JP3217954B2,inf,inf,inf,inf,inf,inf,inf,inf,inf
303542,CN200710023249.4,CN201062869Y,0.393024,0.104634,0.317174,0.145856,0.044098,inf,1.355612,0.018861,inf
303543,CN200710023249.4,US4955359A,0.364394,0.169634,0.199885,0.221151,0.066088,inf,2.178215,0.065236,inf


In [73]:
tfidf_table = pd.read_csv(f'{workdata}/tfidf_table_filter.csv').set_index('Patent_ID')

In [74]:
tfidf_table

,aaa,ab,abil,abnorm,abov,abras,absenc,absorb,absorpt,abstract,...,yield,yoke,zero,zinc,zirconium,zn,zno,zone,zoom,zr
Patent_ID,,,,,,,,,,,,,,,,,,,,,
CN98118142.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN200710126165.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN200610060717.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN00103340.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CN200710075612.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JPH0217849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WO2005017670A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DE202005008280U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074061,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
#### If time permits, do some distance calculations based on TFIDF, then we can use jaccard:
from scipy.spatial.distance import euclidean, cosine, jensenshannon, jaccard
def binarize_features(features, threshold=0):
    return (features > threshold).astype(int)

### Get DIST using TFIDF:
def get_distance_tfidf(pair, tfidf_table, distance_type='euclidean'):
    patent1_id = pair['APPLN_ID_SIPO']
    patent2_id = pair['PubNum_google']
    
    if patent1_id not in tfidf_table.index or patent2_id not in tfidf_table.index:
        return float('inf')  # Return a large distance value if either key is missing
    
    patent1_features = tfidf_table.loc[patent1_id].values
    patent2_features = tfidf_table.loc[patent2_id].values

    if distance_type == 'euclidean':
        return euclidean(patent1_features, patent2_features)
    elif distance_type == 'cosine':
        return cosine(patent1_features, patent2_features)
    elif distance_type == 'jensenshannon':
        return jensenshannon(patent1_features, patent2_features) 
    elif distance_type == 'jaccard':
        patent1_binary_features = binarize_features(patent1_features)
        patent2_binary_features = binarize_features(patent2_features)
        return jaccard(patent1_binary_features, patent2_binary_features)
    else:
        raise ValueError(f"Unsupported distance type: {distance_type}")

        
distance_tfidf_df = pd.DataFrame(columns=['APPLN_ID_SIPO', 'PubNum_google', 
                                    'distance_tfidf_euclidean', 'distance_tfidf_cosine', 
                                    'distance_tfidf_jensenshannon', 'distance_tfidf_jaccard'])


for _, pair in PAIRS.iterrows():
    distance_tfidf_euclidean = get_distance_tfidf(pair, tfidf_table, distance_type='euclidean')
    distance_tfidf_cosine = get_distance_tfidf(pair, tfidf_table, distance_type='cosine')
    distance_tfidf_jensenshannon = get_distance_tfidf(pair, tfidf_table, distance_type='jensenshannon')
    distance_tfidf_jaccard = get_distance_tfidf(pair, tfidf_table, distance_type='jaccard')
   

    result_row = pd.DataFrame({
        'APPLN_ID_SIPO': [pair['APPLN_ID_SIPO']],
        'PubNum_google': [pair['PubNum_google']],
        'distance_tfidf_euclidean': [distance_tfidf_euclidean],
        'distance_tfidf_cosine': [distance_tfidf_cosine],
        'distance_tfidf_jensenshannon': [distance_tfidf_jensenshannon],
        'distance_tfidf_jaccard': [distance_tfidf_jaccard]
    })

    distance_tfidf_df = pd.concat([distance_tfidf_df, result_row], ignore_index=True)
distance_tfidf_df.to_csv(f'{workdata}/distance_tfidf_df.csv', index= False)
distance_tfidf_df

,APPLN_ID_SIPO,PubNum_google,distance_tfidf_euclidean,distance_tfidf_cosine,distance_tfidf_jensenshannon,distance_tfidf_jaccard
0,CN98118142.2,JP2897659B,inf,inf,inf,inf
1,CN98118142.2,US5644677,1.414214,1.0,0.832555,1.0
2,CN98118142.2,KR100188089B,1.408604,0.992082,0.824432,0.980392
3,CN98118142.2,US5572591,1.387102,0.962025,0.812365,0.981481
4,CN98118142.2,JPH09322078,1.411895,0.996724,0.827688,0.984848
...,...,...,...,...,...,...
303540,CN200710023249.4,CN2676102Y,1.266035,0.801423,0.731032,0.872093
303541,CN200710023249.4,JP3217954B2,inf,inf,inf,inf
303542,CN200710023249.4,CN201062869Y,0.694888,0.241435,0.478485,0.605634
303543,CN200710023249.4,US4955359A,0.967471,0.468,0.650876,0.860465


## Sentiment polarity and emotions for each patent
(not used for similarity calculations, but just for course project, so I will just do a sample of 200. \
it can be stored as a variable as calculate the diff for a pair.

In [83]:
def load_emolex(filepath):
    emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], sep='\t')
    return emolex_df

emolex_filepath = 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
emolex_words = load_emolex(emolex_filepath)
emolex_words

,word,emotion,association
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0
...,...,...,...
141535,zoom,negative,0
141536,zoom,positive,0
141537,zoom,sadness,0
141538,zoom,surprise,0


In [95]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [93]:
def calculate_emotions(text, emolex_words):
    emotions_count = {emotion: 0 for emotion in emolex_words.columns[2:]}
    
    tokens = tokenize_and_filter(text)
    
    for token in tokens:
        if token in emolex_words['word'].values:
            word_emotions = emolex_words[emolex_words['word'] == token].iloc[0, 2:]
            for emotion, value in zip(emotions_count.keys(), word_emotions):
                emotions_count[emotion] += value

    return emotions_count

# Function to calculate sentiment polarity
def calculate_sentiment_polarity(text):
    return TextBlob(text).sentiment.polarity


In [99]:
pooling_patent_data = pd.read_csv(f'{workdata}/pooling_patent_data_filter.csv')
pooling_patent_data = pooling_patent_data.sample(200)

In [ ]:
from textblob import TextBlob

# Initialize a list to store the dataframes
dataframes = []

# Iterate through the patents DataFrame
for index, patent in pooling_patent_data.iterrows():
    text = str(patent['Patent_abstract'])
    
    # Calculate sentiment polarity
    sentiment_polarity = calculate_sentiment_polarity(text)
    
    # Calculate emotions
    emotions_count = calculate_emotions(text, emolex_words)

    # Create a DataFrame with the results and append it to the dataframes list
    temp_df = pd.DataFrame({'Patent_ID': [patent['Patent_ID']],
                            'Sentiment_Polarity': [sentiment_polarity],
                            **emotions_count})
    dataframes.append(temp_df)

# Concatenate all the dataframes in the list
sentiment_and_emotions_df = pd.concat(dataframes, ignore_index=True)
sentiment_and_emotions_df.to_csv(f'{workdata}/sentiment_and_emotions_df_sample.csv', index= False)
sentiment_and_emotions_df